<a href="https://colab.research.google.com/github/curt0230/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/c2d4u.te

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-10 21:27:55--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  6.02MB/s    in 0.2s    

2022-07-10 21:27:56 (6.02 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [ ]:
from pyspark.sql.functions import to_date, col
# Read in the Review dataset as a DataFrame
df.printSchema()
df = df.select(col("marketplace"),col("customer_id"),col("review_id"),col("product_id"),col("product_parent"),col("product_title"),col("product_category"),col("star_rating"),col("helpful_votes"),col("total_votes"),col("vine"),col("verified_purchase"),col("review_headline"),col("review_body"),to_date(col("review_date")).alias("review_date"))
df.show(5)
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rat

In [ ]:
# confirm:  more than 20 votes and >50% helpful votes
df_total_votes = df.groupBy("vine").agg({'total_votes': 'sum','helpful_votes': 'sum'}).withColumnRenamed("sum(total_votes)", "sum_total_votes").withColumnRenamed("sum(helpful_votes)","sum_helpful_votes").dropna()
df_total_votes.show()

+----+---------------+-----------------+
|vine|sum_total_votes|sum_helpful_votes|
+----+---------------+-----------------+
|   Y|          23943|            20057|
|   N|        5101480|          4349532|
+----+---------------+-----------------+



In [ ]:
df_total_five_star_votes = df.filter(col("star_rating")==5).groupBy("vine").agg({"total_votes":"sum"}).withColumnRenamed("sum(total_votes)", "sum_total_five_star_votes").withColumnRenamed("vine", "five_star_vine")
df_total_five_star_votes.show()


+--------------+-------------------------+
|five_star_vine|sum_total_five_star_votes|
+--------------+-------------------------+
|             Y|                     8699|
|             N|                  2686517|
+--------------+-------------------------+



In [ ]:
df_totals = df_total_votes.join(df_total_five_star_votes,df_total_votes.vine ==  df_total_five_star_votes.five_star_vine,"inner").select("vine","sum_total_votes","sum_total_five_star_votes")
df_totals.show(truncate=False)

+----+---------------+-------------------------+
|vine|sum_total_votes|sum_total_five_star_votes|
+----+---------------+-------------------------+
|Y   |23943          |8699                     |
|N   |5101480        |2686517                  |
+----+---------------+-------------------------+



In [ ]:
from pyspark.sql.functions import lit

df_overall_votes = df.groupBy().agg({'total_votes': 'sum'}).withColumnRenamed("sum(total_votes)", "sum_total_votes").select(lit("A").alias("vine"),"sum_total_votes").dropna()
df_overall_votes.show()

+----+---------------+
|vine|sum_total_votes|
+----+---------------+
|   A|        5125423|
+----+---------------+



In [ ]:
df_overall_five_star_votes = df.filter(col("star_rating")==5).groupBy().agg({'total_votes': 'sum'}).withColumnRenamed("sum(total_votes)", "sum_total_five_star_votes").select(lit("A").alias("five_star_vine"),"sum_total_five_star_votes").dropna()
df_overall_five_star_votes.show()

+--------------+-------------------------+
|five_star_vine|sum_total_five_star_votes|
+--------------+-------------------------+
|             A|                  2695216|
+--------------+-------------------------+



In [ ]:
df_overall_totals = df_overall_votes.join(df_overall_five_star_votes,df_overall_votes.vine ==  df_overall_five_star_votes.five_star_vine,"inner").select("vine","sum_total_votes","sum_total_five_star_votes")
df_overall_totals.show()

+----+---------------+-------------------------+
|vine|sum_total_votes|sum_total_five_star_votes|
+----+---------------+-------------------------+
|   A|        5125423|                  2695216|
+----+---------------+-------------------------+



In [ ]:
df_totals = df_totals.union(df_overall_totals)
df_totals.show()

+----+---------------+-------------------------+
|vine|sum_total_votes|sum_total_five_star_votes|
+----+---------------+-------------------------+
|   Y|          23943|                     8699|
|   N|        5101480|                  2686517|
|   A|        5125423|                  2695216|
+----+---------------+-------------------------+



In [17]:
df_totals.withColumn("percent_five_star_votes",df_totals["sum_total_five_star_votes"]/df_totals["sum_total_votes"]*100).show()

+----+---------------+-------------------------+-----------------------+
|vine|sum_total_votes|sum_total_five_star_votes|percent_five_star_votes|
+----+---------------+-------------------------+-----------------------+
|   Y|          23943|                     8699|      36.33212212337635|
|   N|        5101480|                  2686517|      52.66152175447125|
|   A|        5125423|                  2695216|     52.585240281631386|
+----+---------------+-------------------------+-----------------------+

